In [1]:
import pandas as pd
import numpy as np
import pickle
import weighted_sentiment_functions_final as wsff
import stock_price_calculation_trials as spct
from datetime import datetime
from tqdm import tqdm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# To do: organise the print statements, write code to save output lists into pickle files.

In [3]:
all_data = pd.read_csv('final_all_mdna_data.csv', sep='.@', header=None, names=['DELETE', 'CIK', 'TICKER', 'DATE', 'TEXT'], dtype={'CIK': object}, engine='python', parse_dates=[3], infer_datetime_format=True, encoding='utf-8')
all_data

,DELETE,CIK,TICKER,DATE,TEXT
0,NaN,0000002488,AMD,2019-02-08,Our 2018 financial results demonstrate the suc...
1,NaN,0000002488,AMD,2018-02-27,2017 was an important year for AMD as we launc...
2,NaN,0000002488,AMD,2017-02-21,As we continued to focus on our strategy to im...
3,NaN,0000002488,AMD,2016-02-18,We faced a challenging business environment in...
4,NaN,0000002488,AMD,2015-02-19,"Net revenue for 2014 was $5.5 billion, an incr..."
...,...,...,...,...,...
233,NaN,0000766704,WELL,2009-03-02,"Health Care REIT, Inc., an S&P 500 company, is..."
234,NaN,0000766704,WELL,2008-02-28,"Health Care REIT, Inc. is an equity real esta..."
235,NaN,0000766704,WELL,2007-03-01,"Health Care REIT, Inc. is a self-administered,..."
236,NaN,0000766704,WELL,2006-03-10,"Health Care REIT, Inc. is a self-administered,..."


In [4]:
new_all_data = all_data.drop(['DELETE'], axis=1)
new_all_data

,CIK,TICKER,DATE,TEXT
0,0000002488,AMD,2019-02-08,Our 2018 financial results demonstrate the suc...
1,0000002488,AMD,2018-02-27,2017 was an important year for AMD as we launc...
2,0000002488,AMD,2017-02-21,As we continued to focus on our strategy to im...
3,0000002488,AMD,2016-02-18,We faced a challenging business environment in...
4,0000002488,AMD,2015-02-19,"Net revenue for 2014 was $5.5 billion, an incr..."
...,...,...,...,...
233,0000766704,WELL,2009-03-02,"Health Care REIT, Inc., an S&P 500 company, is..."
234,0000766704,WELL,2008-02-28,"Health Care REIT, Inc. is an equity real esta..."
235,0000766704,WELL,2007-03-01,"Health Care REIT, Inc. is a self-administered,..."
236,0000766704,WELL,2006-03-10,"Health Care REIT, Inc. is a self-administered,..."


In [5]:
# Check dataframe for any missing values:
# new_all_data.isnull().values.any()
# new_all_data.isnull().sum().sum()

In [6]:
# Comment/Uncomment following 4 lines to retrain the classifier
trained_model, vocab_dict = wsff.train_classifier(classifier_model='Random_Forst')
# save the model and vocab to disk
filename = 'finalised_classifier_model_balanced.sav'
pickle.dump(trained_model, open(filename, 'wb'))

with open('vocab_mapping_dictionary.pkl', 'wb') as f:
    pickle.dump(vocab_dict, f)

[[  0 420]
 [  1 667]
 [  2 887]]
1.23657548126


In [7]:
# load the trained classifier model and vocab from disk
filename = 'finalised_classifier_model_balanced.sav'
model = pickle.load(open(filename, 'rb'))

with open('vocab_mapping_dictionary.pkl', 'rb') as f:
    vocab = pickle.load(f)

To-do: Might need to modify this, instead round each value maybe?
In regular sent analysis you get 0, 0.5, 1. With averaging justifications you get between 0 and 1, i.e. scaled/weighted.
Below is more like weighted mode aggregated. Whereas above 'weighted' function is like weighted mean aggregated.
But mode doesn't make much sense because it gets rid of the weight, so it is more like checking if 5 mined justifications can
capture the full sentiment of the entire document.
Read what you wrote in your thesis and then clarify all these sentiment score terms below!!!!!!

In [8]:
weighted_sentiments = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    weighted_sentiment = wsff.get_weighted_sentiment(row['TEXT'], model, clustering_model='Agglomerative')
    weighted_sentiments.append(weighted_sentiment)
print(weighted_sentiments)
new_all_data['weighted_sentiments'] = weighted_sentiments

unweighted_sentiments = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    unweighted_sentiment = wsff.get_unweighted_sentiment(row['TEXT'], model, clustering_model='Agglomerative')
    unweighted_sentiments.append(unweighted_sentiment)
print(unweighted_sentiments)
new_all_data['unweighted_sentiments'] = unweighted_sentiments

weighted_sentiments_DBSCAN = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    weighted_sentiment = wsff.get_weighted_sentiment(row['TEXT'], model, clustering_model='DBSCAN')
    weighted_sentiments_DBSCAN.append(weighted_sentiment)
print(weighted_sentiments_DBSCAN)
new_all_data['weighted_sentiments_DBSCAN'] = weighted_sentiments_DBSCAN


weighted_sentiments_MeanShift = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    weighted_sentiment = wsff.get_weighted_sentiment(row['TEXT'], model, clustering_model='MeanShift')
    weighted_sentiments_MeanShift.append(weighted_sentiment)
print(weighted_sentiments_MeanShift)
new_all_data['weighted_sentiments_MeanShift'] = weighted_sentiments_MeanShift

Getting sentiment scores: 0Filings [00:00, ?Filings/s]

[[  0 420]
 [  1 607]
 [  2 887]]
1.24399164054
token ids: [[366, 61, 233, 32, 1947, 225, 158, 61, 233, 225, 32, 5200], [366, 434, 278, 2583, 366, 1602, 877, 4355], [1082, 6362, 2911, 366, 434, 1268, 278, 364, 12, 5164, 1404, 765, 12, 203, 5949, 3997, 366, 434, 1853, 586, 4436, 2353, 1977, 1434, 434, 1853, 41, 44, 44, 5155, 5167, 12, 1208, 1434, 586, 2207, 4436, 610, 3554, 2890, 232, 2920, 366, 434, 1853], [4545, 1332, 256, 2022, 227, 228, 819, 119, 858, 254, 158, 284, 254, 227, 228, 220, 5200], [1082, 2192, 2520, 366, 3460, 3564, 86, 16, 2489, 366, 395, 500, 607, 696, 3564, 32, 1893]]
X_test [[1082, 6362, 2911, 366, 434, 1268, 278, 364, 12, 5164, 1404, 765, 12, 203, 5949, 3997, 366, 434, 1853, 586, 4436, 2353, 1977, 1434, 434, 1853, 41, 44, 44, 5155, 5167, 12, 1208, 1434, 586, 2207, 4436, 610, 3554, 2890, 232, 2920, 366, 434, 1853], [4545, 1332, 256, 2022, 227, 228, 819, 119, 858, 254, 158, 284, 254, 227, 228, 220, 5200], [1082, 2192, 2520, 366, 3460, 3564, 86, 16, 2489, 366, 395, 500

Getting sentiment scores: 1Filings [00:05,  5.35s/Filings]

[[  0 420]
 [  1 636]
 [  2 887]]
1.24034997427
token ids: [[1082, 345, 5892, 215, 3966, 366, 2204, 577, 2786, 229], [1082, 6362, 2911, 366, 434, 1268, 278, 364, 12, 5164, 1404, 765, 12, 203, 5949, 3997, 366, 434, 1853, 586, 4436, 2353, 1977, 1434, 434, 1853, 41, 44, 44, 5155, 5167, 12, 1208, 1434, 586, 2207, 4436, 610, 3554, 2890, 232, 2920, 366, 434, 1853], [17, 3564, 13, 97, 5894, 841, 336, 6042, 1534, 2813, 2874, 16, 4519, 5507, 32, 1898, 1319, 2533, 3964, 838], [1082, 2973, 366, 3460, 3564, 488, 5200, 2133, 2348, 1335, 1319, 6982, 393, 1177], [366, 61, 233, 32, 5200, 1947, 225, 158, 216, 61, 53, 225, 32, 5739]]
X_test [[1082, 6362, 2911, 366, 434, 1268, 278, 364, 12, 5164, 1404, 765, 12, 203, 5949, 3997, 366, 434, 1853, 586, 4436, 2353, 1977, 1434, 434, 1853, 41, 44, 44, 5155, 5167, 12, 1208, 1434, 586, 2207, 4436, 610, 3554, 2890, 232, 2920, 366, 434, 1853], [17, 3564, 13, 97, 5894, 841, 336, 6042, 1534, 2813, 2874, 16, 4519, 5507, 32, 1898, 1319, 2533, 3964, 838]]


Getting sentiment scores: 2Filings [00:08,  4.14s/Filings]

[[  0 420]
 [  1 681]
 [  2 887]]
1.23490945674
token ids: [[357, 5739, 1082, 1202, 3093, 1628, 366, 61, 2543], [345, 889, 5739, 1082, 2973, 17, 1000, 770, 2923, 16, 216, 3569, 2166, 678, 6714, 1319, 32, 6982, 1512], [1082, 345, 4436, 366, 4098, 287, 4570, 32, 256, 813, 225, 5925, 3425, 3088, 309, 1249, 45, 5185, 4436, 4350], [1675, 2348, 366, 5894, 229, 632, 2133, 566, 1335, 2069, 336, 87], [61, 53, 32, 5739, 119, 225, 158, 225, 32]]
X_test [[345, 889, 5739, 1082, 2973, 17, 1000, 770, 2923, 16, 216, 3569, 2166, 678, 6714, 1319, 32, 6982, 1512], [1082, 345, 4436, 366, 4098, 287, 4570, 32, 256, 813, 225, 5925, 3425, 3088, 309, 1249, 45, 5185, 4436, 4350]]


Getting sentiment scores: 3Filings [00:11,  3.83s/Filings]

[[  0 420]
 [  1 647]
 [  2 887]]
1.23899692938
token ids: [[13, 841, 215, 395, 500, 770, 80, 776, 5894, 32, 770], [1082, 345, 2520, 366, 2786, 1923, 57, 7164, 2133, 32, 607, 2069, 3474], [366, 61, 2543, 1908, 857, 254, 41, 1031, 254, 4511], [1082, 345, 1860, 25, 41, 366, 1826, 848, 740, 87, 3877, 2489, 1458, 2800, 291], [1082, 5677, 4473, 6, 278, 589, 1926, 2316, 61, 2543, 813, 930, 225, 5739]]
X_test []


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
full_sentiments_mean = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mean = wsff.get_full_sentiment_mean(row['TEXT'], model)
    full_sentiments_mean.append(full_sentiment_mean)
print(full_sentiments_mean)
new_all_data['full_mean'] = full_sentiments_mean

full_sentiments_mean_unrounded = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mean_unrounded = wsff.get_full_sentiment_mean_unrounded(row['TEXT'], model)
    full_sentiments_mean_unrounded.append(full_sentiment_mean_unrounded)
print(full_sentiments_mean_unrounded)
new_all_data['full_mean_unrounded'] = full_sentiments_mean_unrounded

full_sentiments_mode = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mode = wsff.get_full_sentiment_mode(row['TEXT'], model)
    full_sentiments_mode.append(full_sentiment_mode)
print(full_sentiments_mode)
new_all_data['full_mode'] = full_sentiments_mode

Note the time difference in weighted vs full sentiment scores:
weighted:
Getting sentiment scores: 239Filings [09:29,  2.38s/Filings]
Getting sentiment scores: 239Filings [09:13,  2.32s/Filings]

Full:
Getting sentiment scores: 239Filings [00:04, 51.05Filings/s]
Getting sentiment scores: 239Filings [00:04, 50.69Filings/s]
Getting sentiment scores: 239Filings [00:04, 50.40Filings/s]

Likely caused by the embeddings extraction time.

In [ ]:
new_all_data.head()

Trial outputs for all the below stock calculations:
1:
[23.049999237060547, 331.04998779296875, 26.155519485473633]
[Timestamp('2019-02-08 00:00:00'), Timestamp('2019-01-31 00:00:00'), Timestamp('2010-11-18 00:00:00')]

2:
[52.259998321533203, 517.46002197265625, 16.585308074951172]
[Timestamp('2020-02-10 00:00:00'), Timestamp('2020-01-31 00:00:00'), Timestamp('2011-11-18 00:00:00')]
[27.090000152587891, 371.19000244140625, 27.930322647094727]
[Timestamp('2019-05-08 00:00:00'), Timestamp('2019-04-30 00:00:00'), Timestamp('2011-02-18 00:00:00')]

3:
[126.72451215316249, 56.308727096605168, -36.589643787566928]
[17.527119519516933, 12.125061509907129, 6.7855779450558877]

In [ ]:
def get_current_stock_price_date(ticker, date):
    given_date = date
    while True:
        try:
            stock_price = spct.get_stock_adj_close(ticker, given_date)
            break
        except KeyError:
            given_date = spct.add_days(given_date,1)
    return given_date, stock_price

stock_prices = []
given_dates = []


    
for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], row['DATE'])
    stock_prices.append(stock_price)
    given_dates.append(given_date)
    
print(stock_prices)
print(given_dates)

In [ ]:
# Check to make sure it isn't SBUX out of date 2019, index 209 should now return SBUX 2018-11-16
print(new_all_data.iloc[209])
print(given_dates[209])
date_one_yearsbux = spct.add_years(given_dates[209], 1)
print(date_one_yearsbux) # Should be 2019-11-16

In [ ]:
# To-do: create 6 month window as well.
# Getting an error at index 209: issue was SBUX 2019 is in Nov, so next year is Nov 2020, hasn't happened yet.
stock_prices_next_year = []
given_dates_next_year = []
stock_prices_three_months = []
given_dates_three_months = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    date_one_year = spct.add_years(given_dates[i], 1)
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], date_one_year)
    stock_prices_next_year.append(stock_price)
    given_dates_next_year.append(given_date)
    
for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    date_three_months = spct.add_months(given_dates[i], 3)
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], date_three_months)
    stock_prices_three_months.append(stock_price)
    given_dates_three_months.append(given_date)

print(stock_prices_next_year)
print(given_dates_next_year)
print(stock_prices_three_months)
print(given_dates_three_months)

In [ ]:
percent_changes_one_year = []
percent_changes_three_months = []

for old, new in zip(stock_prices, stock_prices_next_year):
    change = spct.percent_change(old,new)
    percent_changes_one_year.append(change)
    
for old, new in zip(stock_prices, stock_prices_three_months):
    change = spct.percent_change(old,new)
    percent_changes_three_months.append(change)

new_all_data['% change 1 year'] = percent_changes_one_year
new_all_data['% change 3 months'] = percent_changes_three_months

print(percent_changes_one_year)
print(percent_changes_three_months)

In [ ]:
new_all_data.head()
# To-do balance the training data to have less neutral labels

In [ ]:
new_all_data.to_pickle("./full_data_df_1.pkl")

In [ ]:
# Name used in the functions below is still new_all_data, so maybe change the name below when reopening this file.
new_all_data = pd.read_pickle("./full_data_df_1.pkl")
new_all_data.head()

Choosing spearmean versus pearson correlation:
'I noted in a comment to the post that usually spearmanr() is used for ranks and it does not seem like you are correlating ranked data. I'm making that assessment based on non-integral values of the 2 sequences-usually ranks are non-negative integral values.

Given the second issue (as it seems) I'd recommend using pearsonr(). Provided your version of scipy is 1.3.X or greater, the error message is informative and tells you exactly this issue:'

Note: Set p-value to 0.05 or less to be statistically significant. 
'The null hypothesis is that the two variables are uncorrelated. The p-value is a number between zero and one that represents the probability that your data would have arisen if the null hypothesis were true.'

In [ ]:
# This block is for checking logit outputs on individual columns
# from scipy import special, stats

# normalised_sentiments_weighted = special.logit(new_all_data['full_mean_unrounded'])
# normalised_sentiments_weighted_fixed = np.nan_to_num(normalised_sentiments_weighted)
# # normalised_sentiments_weighted.values
# normalised_sentiments_weighted_fixed
# # np.isnan(normalised_sentiments_weighted_fixed).any()
# # np.isinf(normalised_sentiments_weighted_fixed).any()

In [ ]:
# To-do: format printed lines with % stuff to make it more organised, google how to print with even spacing between
# To-do: Reinclude MeanShift in the list below after restarting kernel with updated .py imports
# To-do: Maybe calculate similarity scores between weighted sentiments lists and full sentiments lists
# Using np.nan_to_num() to solve the infs error. Check the make sure the maths for this is okay
from scipy import special, stats


def logit_correlate(sentiments, percent_changes):
    normalised_sentiments = special.logit(sentiments)
#     print(normalised_sentiments.values)
    normalised_sentiments_fixed = np.nan_to_num(normalised_sentiments)
#     print(np.isnan(normalised_sentiments_fixed).any())
#     print(np.isinf(normalised_sentiments_fixed).any())
#     print(np.isnan(percent_changes.values).any())
#     print(np.isinf(percent_changes.values).any())
    r, p_value = stats.pearsonr(normalised_sentiments_fixed, percent_changes)
    return r, p_value

# for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'weighted_sentiments_DBSCAN', 'full_mean', 'full_mean_unrounded', 'full_mode']:
#     print(sentiment)
for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'weighted_sentiments_DBSCAN', 'weighted_sentiments_MeanShift', 'full_mean', 'full_mean_unrounded', 'full_mode']:
    r_1yr, p_value_1yr = stats.pearsonr(new_all_data[sentiment], new_all_data['% change 1 year'])
#     print(sentiment, '1 year', 'r=', r_1yr, 'p=', p_value_1yr)
    time = '1 year'
    print(f'{sentiment:30} {time:10} r={r_1yr:<25} p={p_value_1yr}')
print('\n')

# for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'weighted_sentiments_DBSCAN', 'full_mean', 'full_mean_unrounded', 'full_mode']:
for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'weighted_sentiments_DBSCAN', 'weighted_sentiments_MeanShift', 'full_mean', 'full_mean_unrounded', 'full_mode']:
    r_3month, p_value_3month = stats.pearsonr(new_all_data[sentiment], new_all_data['% change 3 months'])
#     print(sentiment, '3 months', 'r=', r_3month, 'p=', p_value_3month)
    time = '3 months'
    print(f'{sentiment:30} {time:10} r={r_3month:<25} p={p_value_3month}')
    

In [ ]:
# Results from correlation first attempt:

# weighted_sentiments            1 year     r=-0.050994704451746364     p=0.4335826338188967
# unweighted_sentiments          1 year     r=nan                       p=nan
# weighted_sentiments_DBSCAN     1 year     r=-0.06924535039202072      p=0.2873655870349446
# weighted_sentiments_MeanShift  1 year     r=-0.04549583883503607      p=0.4848381950662208
# full_mean                      1 year     r=-0.07114038012496435      p=0.27434649762809193
# full_mean_unrounded            1 year     r=-0.08882127181888153      p=0.1720160449195358
# full_mode                      1 year     r=nan                       p=nan


# weighted_sentiments            3 months   r=-0.07086475034518555      p=0.27621481315395385
# unweighted_sentiments          3 months   r=nan                       p=nan
# weighted_sentiments_DBSCAN     3 months   r=-0.04400432937353048      p=0.49928259331642083
# weighted_sentiments_MeanShift  3 months   r=-0.003751647475437115     p=0.9540886896304036
# full_mean                      3 months   r=-0.0534469143564483       p=0.4117689600688962
# full_mean_unrounded            3 months   r=-0.12512998328588287      p=0.053876058043276866
# full_mode                      3 months   r=nan                       p=nan

In [ ]:
# To-do: maybe leave x axis unlabelled as it means nothing.
import numpy as np
import matplotlib.pyplot as plt

t = new_all_data.index
data1 = new_all_data['weighted_sentiments']
data2 = new_all_data['% change 1 year']

fig, ax1 = plt.subplots()

color = 'tab:purple'
ax1.set_xlabel('index')
ax1.set_ylabel('sentiment scores', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim(0, 1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('% changes in stock price', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
t = new_all_data.index
data1 = new_all_data['weighted_sentiments_DBSCAN']
data2 = new_all_data['% change 1 year']

fig, ax1 = plt.subplots()

color = 'tab:purple'
ax1.set_xlabel('index')
ax1.set_ylabel('sentiment scores', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim(0, 1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('% changes in stock price', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()